In [5]:
#TASK 1 
#Use Threads to create an image download server 
#Use this URL - https://source.unsplash.com/user/c_v_r/200x200
#to download 500 random images 
#Use atleast 3 (or more) threads to perform the download operation in parallel. 
#It is okay to replace the image (with same filename)

import requests


def download_img(img_url: str):
    
    res = requests.get(img_url, stream=True)
   
    filename = "./abc.jpg" #OR "./foldername/abc.jpg" You can also make a folder and save images in the folder

    with open(filename, 'wb') as f:
        for block in res.iter_content(1024):
            f.write(block)

#download_img("https://source.unsplash.com/user/c_v_r/200x200")


In [6]:
import threading
import requests

def download_img():
    
    img_url = "https://source.unsplash.com/user/c_v_r/200x200"
    for _ in range(1):
        res = requests.get(img_url, stream=True)
        filename = "./abc.jpg" #OR "./foldername/abc.jpg" You can also make a folder and save images in the folder

        with open(filename, 'wb') as f:
            for block in res.iter_content(1024):
                f.write(block)
            
    
def main_task():
    
    t = []
    
    for i in range(5):
        t.append(threading.Thread(target=download_img))
    
    
    # start threads
    for i in range(5):
        t[i].start()
   
   
    for i in range(5):
        t[i].join()
  
main_task()

In [7]:
#TASK 2
#Use a "counter" which is accessed with locks to get the image-number
#This way all downloaded images have a unique number while saving
#File names can now be "img" + counter + ".jpg"

import threading
import requests

counter = 0


def download_img(lock):
    
    global counter

    img_url = "https://source.unsplash.com/user/c_v_r/200x200"
    for _ in range(10):
       
        
        res = requests.get(img_url, stream=True)
        
        lock.acquire()
        counter = counter +1
        filename = "./folder/a"+str(counter)+".jpg" 
        lock.release()
        

        with open(filename, 'wb') as f:
            for block in res.iter_content(1024):
                f.write(block)
        
         
    
def main_task():
    
    global counter
    lock = threading.Lock()
    t = []
    
    for i in range(5):
        t.append(threading.Thread(target=download_img,args=(lock,)))
    
    
    # start threads
    for i in range(5):
        t[i].start()
   
   
    for i in range(5):
        t[i].join()
  
main_task()

In [ ]:
#TASK 3
# Simulate a deadlock when one of the thread throws an error without releasing the lock

import threading
import requests

counter = 0


def download_img(lock):
    
    global counter

    img_url = "https://source.unsplash.com/user/c_v_r/200x200"
    for _ in range(100):
       
        
        res = requests.get(img_url, stream=True)
        
        lock.acquire()
        counter = counter +1
        filename = "./images/a"+str(counter)+".jpg" 
        if(counter == 1):
            raise Exception("Random Error")
            #Error only in one thread -- but because others are waiting for the lock, 
            #The lock was never released by the thread with the error, so others are still waiting
        lock.release()
        

        with open(filename, 'wb') as f:
            for block in res.iter_content(1024):
                f.write(block)
        
         
    
def main_task():
    
    global counter
    lock = threading.Lock()
    t = []
    
    for i in range(5):
        t.append(threading.Thread(target=download_img,args=(lock,)))
    
    
    # start threads
    for i in range(5):
        t[i].start()
   
   
    for i in range(5):
        t[i].join()
  
main_task()



Exception in thread Thread-33:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-e95bbda2a84b>", line 24, in download_img
Exception: Random Error


In [ ]:

#TASK 4
#Remove the deadlock
#Make sure all the lock accesses are within a "try" block.
#all the locks should be released in "finally" block. 


import threading
import requests

counter = 0


def download_img(lock):
    
    global counter

    img_url = "https://source.unsplash.com/user/c_v_r/200x200"
    for _ in range(100):
       
        
        res = requests.get(img_url, stream=True)
        
        try:
            lock.acquire()
            counter = counter +1
            filename = "./images/a"+str(counter)+".jpg" 
            if(counter == 1):
                raise Exception("Random Error")
            pass
        except:
            #do nothing
            pass
        finally:
            lock.release()
        

        with open(filename, 'wb') as f:
            for block in res.iter_content(1024):
                f.write(block)
        
         
    
def main_task():
    
    global counter
    lock = threading.Lock()
    t = []
    
    for i in range(5):
        t.append(threading.Thread(target=download_img,args=(lock,)))
    
    
    # start threads
    for i in range(5):
        t[i].start()
   
   
    for i in range(5):
        t[i].join()
  
main_task()




In [ ]:
import multiprocess
import urllib
import time

def download_img(counter,lock):
    
    img_url = "https://source.unsplash.com/user/c_v_r/200x200"
    #filename = "./images/a"+str(counter.value)+".jpg" 
    for _ in range(10):
        
        lock.acquire()
        
        counter.value = counter.value + 1
        print(counter.value)
        
        filename = "./images/a"+str(counter.value)+".jpg" 
        urllib.request.urlretrieve(img_url, filename)
        #res = requests.get(,stream=True,timeout = 10)
        
        
        #with open(filename, 'wb') as f:
        #    for block in res.iter_content(1024):
        #        f.write(block)
        lock.release()
    
        
        
         
    
def main_task():
    
    lock = multiprocess.Lock()
    counter = multiprocess.Value('i', 0)
   
    
    
    p = []
    
    for i in range(5):
        p.append(multiprocess.Process(target=download_img,args=(counter,lock,) ))
    
    
    # start threads
    for i in range(5):
        p[i].start()
   
   
    for i in range(5):
        p[i].join()
  

main_task()



